In [ ]:
#@title Download data from GCP bucket
import sys
!pip install fair-esm
!pip install --upgrade fair-esm
!pip install biopython
if 'google.colab' in sys.modules:
  !gsutil -m cp -r gs://indaba-data .
else:
  !mkdir -p indaba-data/train
  !wget -P indaba-data/train https://storage.googleapis.com/indaba-data/train/train.csv --continue
  !wget -P indaba-data/train https://storage.googleapis.com/indaba-data/train/train_mut.pt --continue
  !wget -P indaba-data/train https://storage.googleapis.com/indaba-data/train/train_wt.pt --continue

  !mkdir -p indaba-data/test
  !wget -P indaba-data/test https://storage.googleapis.com/indaba-data/test/test.csv --continue
  !wget -P indaba-data/test https://storage.googleapis.com/indaba-data/test/test_mut.pt --continue
  !wget -P indaba-data/test https://storage.googleapis.com/indaba-data/test/test_wt.pt --continue

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 5.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 48.3 MB/s eta 0:00:00
Copying gs://indaba-data/README.txt...
Copying gs://indaba-data/test/test.csv...
Copying gs://indaba-data/test/test_wt.pt...
Copying gs://indaba-data/test/test_mut.pt...
Copying gs://indaba-data/train/train.csv...
Copying gs://indaba-data/train/train_mut.pt...
Copying gs://indaba-data/train/train_wt.pt...
==> NOTE: You are downloading one or more large file(s), which would
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crc

In [ ]:
#@title Imports and moving to working directory
import torch 
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf

# move to data folder
%cd indaba-data

/content/indaba-data


In [ ]:
# Load Embedding tensors & Traing csv
# Embeddings were calculated using the ESM 650M pretrained model 
# Tensor shape of embedded data:  [data_len,1280] 
# There are no sequences in the Embedding tensors as we've performed an average of it (torch.mean(embed, dim=1))
# More details in https://huggingface.co/facebook/esm2_t33_650M_UR50D

wt_emb = torch.load("train/train_wt.pt")
mut_emb = torch.load("train/train_mut.pt")
train = pd.read_csv("train/train.csv")
test = pd.read_csv("test/test.csv")

train

,ID,pdb_id,mutation,wt_aa,mutation_pos,mut_aa,wt_seq,mut_seq,ddg
0,0,1GYZ,W1Q,W,1,Q,WIARINAAVRAYGLNYSTFINGLKKAGIELDRKILADMAVRDPQAF...,QIARINAAVRAYGLNYSTFINGLKKAGIELDRKILADMAVRDPQAF...,0.228775
1,1,1GYZ,W1E,W,1,E,WIARINAAVRAYGLNYSTFINGLKKAGIELDRKILADMAVRDPQAF...,EIARINAAVRAYGLNYSTFINGLKKAGIELDRKILADMAVRDPQAF...,0.496896
2,2,1GYZ,W1N,W,1,N,WIARINAAVRAYGLNYSTFINGLKKAGIELDRKILADMAVRDPQAF...,NIARINAAVRAYGLNYSTFINGLKKAGIELDRKILADMAVRDPQAF...,0.163002
3,3,1GYZ,W1H,W,1,H,WIARINAAVRAYGLNYSTFINGLKKAGIELDRKILADMAVRDPQAF...,HIARINAAVRAYGLNYSTFINGLKKAGIELDRKILADMAVRDPQAF...,0.209013
4,4,1GYZ,W1D,W,1,D,WIARINAAVRAYGLNYSTFINGLKKAGIELDRKILADMAVRDPQAF...,DIARINAAVRAYGLNYSTFINGLKKAGIELDRKILADMAVRDPQAF...,0.407602
...,...,...,...,...,...,...,...,...,...
339773,339773,r7-562-TrROS-Hall,V48W,V,48,W,MKKYKITVYDEKTGEKHTIEIEMSEEELEELAKKLAEKHNVKVRIEKV,MKKYKITVYDEKTGEKHTIEIEMSEEELEELAKKLAEKHNVKVRIEKW,-1.614409
339774,339774,r7-562-TrROS-Hall,V48Y,V,48,Y,MKKYKITVYDEKTGEKHTIEIEMSEEELEELAKKLAEKHNVKVRIEKV,MKKYKITVYDEKTGEKHTIEIEMSEEELEELAKKLAEKHNVKVRIEKY,-1.370771
339775,339775,r7-562-TrROS-Hall,V48F,V,48,F,MKKYKITVYDEKTGEKHTIEIEMSEEELEELAKKLAEKHNVKVRIEKV,MKKYKITVYDEKTGEKHTIEIEMSEEELEELAKKLAEKHNVKVRIEKF,-1.370829
339776,339776,r7-562-TrROS-Hall,V48P,V,48,P,MKKYKITVYDEKTGEKHTIEIEMSEEELEELAKKLAEKHNVKVRIEKV,MKKYKITVYDEKTGEKHTIEIEMSEEELEELAKKLAEKHNVKVRIEKP,-0.757622


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from Bio.Seq import Seq
train_data, test_data = train_test_split(train, test_size=0.2, random_state=42)
# Further split the train set into train and validation sets
train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)

In [ ]:
def preprocess_data(data):
    preprocessed_data = []
    for row in data.itertuples(index=False):
        wild_type = row[0]
        mutated = row[1]

        # Convert the sequences to Bio.Seq objects
        wild_type_seq = Seq(wild_type)
        mutated_seq = Seq(mutated)

        # Append the preprocessed data as a tuple to the list
        preprocessed_data.append((str(wild_type_seq), str(mutated_seq)))

    return preprocessed_data

In [ ]:
X_train = preprocess_data(train_data[["mut_aa"	,"wt_seq","ddg"]])
y_train = train_data["ddg"]

X_val = preprocess_data(val_data[["mut_aa"	,"wt_seq","ddg"]])
y_val = val_data["ddg"]

X_test = preprocess_data(test_data[["mut_aa"	,"wt_seq","ddg"]])
y_test = test_data["ddg"]

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 57.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 104.0 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer
# Load the pre-trained tokenizer
tokenizer = AutoTokenizer.from_pretrained('facebook/esm2_t33_650M_UR50D')

In [ ]:
X_train_encoded = tokenizer(X_train, padding=True, truncation=True, max_length=100)
X_val_encoded = tokenizer(X_val, padding=True, truncation=True, max_length=100)
X_test_encoded = tokenizer(X_test, padding=True, truncation=True, max_length=100)

In [ ]:
# Assuming you have defined the fine-tuning architecture and layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from transformers import TFEsmForSequenceClassification
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
with tf.device("/gpu:0"):
             model = TFEsmForSequenceClassification.from_pretrained('facebook/esm2_t33_650M_UR50D')
# Compile the model with a suitable optimizer, loss function, and evaluation metric
model.compile(optimizer=Adam(learning_rate=1e-5), loss=MeanSquaredError(), metrics=['accuracy'])

Some layers from the model checkpoint at facebook/esm2_t33_650M_UR50D were not used when initializing TFEsmForSequenceClassification: ['lm_head']
- This IS expected if you are initializing TFEsmForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFEsmForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFEsmForSequenceClassification were not initialized from the model checkpoint at facebook/esm2_t33_650M_UR50D and are newly initialized: ['classifier', 'esm/contact_head/regression/kernel:0', 'esm/contact_head/regression/bias:0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import numpy 
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# define batch size and number of epochs
batch_size = 8
epochs = 3
# Preprocess and encode the data
X_train_encoded_np = np.array(X_train_encoded['input_ids'])
X_val_encoded_np = np.array(X_val_encoded['input_ids'])
X_test_encoded_np = np.array(X_test_encoded['input_ids'])
# Train the model
with tf.device("/gpu:0"):
                history = model.fit(X_train_encoded_np, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_val_encoded_np, y_val))

Epoch 1/3


22288/27183 [=======================>......] - ETA: 1:18:58 - loss: 0.9183 - accuracy: 0.0000e+00

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse'])

# Print the model summary
model.summary()

## Prediction & submission

In [ ]:
df_result = pd.DataFrame()
with torch.no_grad():
  for batch_idx, (data_mut,data_wt , target) in tqdm(enumerate(test_dataloader)):
    x1 = data_wt.to(device)
    x2 = data_mut.to(device)
    id = target.to(device)
    # make prediction
    y_pred = model(x1,x2)
    df_result = pd.concat([df_result, pd.DataFrame({"ID":id.squeeze().cpu().numpy().astype(int) , "ddg" : y_pred.squeeze().cpu().numpy()})])

In [ ]:
df_result.to_csv("submission.csv",index=False)